# Title: Medivise with Machine Learning

# Description:
Welcome to our cutting-edge Personalized Medical Recommendation System, a powerful platform designed to assist users in understanding and managing their health. Leveraging the capabilities of machine learning, our system analyzes user-input symptoms to predict potential diseases accurately.

In [ ]:
# libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# for models
from sklearn.svm import SVC
from textblob import TextBlob
from collections import defaultdict
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix

# Load Data sets

In [2]:
# load dataset 
symtoms_df = pd.read_csv('dataset/symtoms_df.csv')
Symptom_severity_df = pd.read_csv('dataset/Symptom-severity.csv')
medications_df = pd.read_csv('dataset/medications.csv')
description_df = pd.read_csv('dataset/description.csv')
diets_df = pd.read_csv('dataset/diets.csv')
precautions_df = pd.read_csv('dataset/precautions_df.csv')
workout_df = pd.read_csv('dataset/workout_df.csv')
Training_df = pd.read_csv('dataset/Training.csv')

# Training Dataset

In [3]:
# load dataset & tools
Training_df.head(3)

itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  shivering  \
0        1          1                     1                    0          0   
1        0          1                     1                    0          0   
2        1          0                     1                    0          0   

   chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  ...  \
0       0           0             0        0                 0  ...   
1       0           0             0        0                 0  ...   
2       0           0             0        0                 0  ...   

   blackheads  scurring  skin_peeling  silver_like_dusting  \
0           0         0             0                    0   
1           0         0             0                    0   
2           0         0             0                    0   

   small_dents_in_nails  inflammatory_nails  blister  red_sore_around_nose  \
0                     0                   0        0                     0   
1                     0                   0        0                     0   
2                     0                   0        0                     0   

   yellow_crust_ooze         prognosis  
0                  0  Fungal infection  
1                  0  Fungal infection  
2                  0  Fungal infection  

[3 rows x 133 columns]

In [4]:
Training_df.shape

(4920, 133)

In [5]:
Training_df.isnull().sum()

itching                 0
skin_rash               0
nodal_skin_eruptions    0
continuous_sneezing     0
shivering               0
                       ..
inflammatory_nails      0
blister                 0
red_sore_around_nose    0
yellow_crust_ooze       0
prognosis               0
Length: 133, dtype: int64

In [6]:
Training_df.describe()

itching    skin_rash  nodal_skin_eruptions  continuous_sneezing  \
count  4920.000000  4920.000000           4920.000000          4920.000000   
mean      0.137805     0.159756              0.021951             0.045122   
std       0.344730     0.366417              0.146539             0.207593   
min       0.000000     0.000000              0.000000             0.000000   
25%       0.000000     0.000000              0.000000             0.000000   
50%       0.000000     0.000000              0.000000             0.000000   
75%       0.000000     0.000000              0.000000             0.000000   
max       1.000000     1.000000              1.000000             1.000000   

         shivering       chills   joint_pain  stomach_pain      acidity  \
count  4920.000000  4920.000000  4920.000000   4920.000000  4920.000000   
mean      0.021951     0.162195     0.139024      0.045122     0.045122   
std       0.146539     0.368667     0.346007      0.207593     0.207593   
min       0.000000     0.000000     0.000000      0.000000     0.000000   
25%       0.000000     0.000000     0.000000      0.000000     0.000000   
50%       0.000000     0.000000     0.000000      0.000000     0.000000   
75%       0.000000     0.000000     0.000000      0.000000     0.000000   
max       1.000000     1.000000     1.000000      1.000000     1.000000   

       ulcers_on_tongue  ...  pus_filled_pimples   blackheads     scurring  \
count       4920.000000  ...         4920.000000  4920.000000  4920.000000   
mean           0.021951  ...            0.021951     0.021951     0.021951   
std            0.146539  ...            0.146539     0.146539     0.146539   
min            0.000000  ...            0.000000     0.000000     0.000000   
25%            0.000000  ...            0.000000     0.000000     0.000000   
50%            0.000000  ...            0.000000     0.000000     0.000000   
75%            0.000000  ...            0.000000     0.000000     0.000000   
max            1.000000  ...            1.000000     1.000000     1.000000   

       skin_peeling  silver_like_dusting  small_dents_in_nails  \
count   4920.000000          4920.000000           4920.000000   
mean       0.023171             0.023171              0.023171   
std        0.150461             0.150461              0.150461   
min        0.000000             0.000000              0.000000   
25%        0.000000             0.000000              0.000000   
50%        0.000000             0.000000              0.000000   
75%        0.000000             0.000000              0.000000   
max        1.000000             1.000000              1.000000   

       inflammatory_nails      blister  red_sore_around_nose  \
count         4920.000000  4920.000000           4920.000000   
mean             0.023171     0.023171              0.023171   
std              0.150461     0.150461              0.150461   
min              0.000000     0.000000              0.000000   
25%              0.000000     0.000000              0.000000   
50%              0.000000     0.000000              0.000000   
75%              0.000000     0.000000              0.000000   
max              1.000000     1.000000              1.000000   

       yellow_crust_ooze  
count        4920.000000  
mean            0.023171  
std             0.150461  
min             0.000000  
25%             0.000000  
50%             0.000000  
75%             0.000000  
max             1.000000  

[8 rows x 132 columns]

In [7]:
Training_df.prognosis.value_counts()

Fungal infection                           120
Hepatitis C                                120
Hepatitis E                                120
Alcoholic hepatitis                        120
Tuberculosis                               120
Common Cold                                120
Pneumonia                                  120
Dimorphic hemmorhoids(piles)               120
Heart attack                               120
Varicose veins                             120
Hypothyroidism                             120
Hyperthyroidism                            120
Hypoglycemia                               120
Osteoarthristis                            120
Arthritis                                  120
(vertigo) Paroymsal  Positional Vertigo    120
Acne                                       120
Urinary tract infection                    120
Psoriasis                                  120
Hepatitis D                                120
Hepatitis B                                120
Allergy      

In [8]:
len(Training_df['prognosis'].unique())

41

In [9]:
Training_df['prognosis'].unique()

array(['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis',
       'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes ',
       'Gastroenteritis', 'Bronchial Asthma', 'Hypertension ', 'Migraine',
       'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice',
       'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia',
       'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins',
       'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia',
       'Osteoarthristis', 'Arthritis',
       '(vertigo) Paroymsal  Positional Vertigo', 'Acne',
       'Urinary tract infection', 'Psoriasis', 'Impetigo'], dtype=object)

# ML Model

# Train Test Split

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [11]:
X = Training_df.drop('prognosis', axis=1)
y = Training_df['prognosis']
y

0                              Fungal infection
1                              Fungal infection
2                              Fungal infection
3                              Fungal infection
4                              Fungal infection
                         ...                   
4915    (vertigo) Paroymsal  Positional Vertigo
4916                                       Acne
4917                    Urinary tract infection
4918                                  Psoriasis
4919                                   Impetigo
Name: prognosis, Length: 4920, dtype: object

In [12]:
# ecoding prognonsis
le = LabelEncoder()
le.fit(y)
Y = le.transform(y)
Y

array([15, 15, 15, ..., 38, 35, 27])

In [13]:
X

itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  \
0           1          1                     1                    0   
1           0          1                     1                    0   
2           1          0                     1                    0   
3           1          1                     0                    0   
4           1          1                     1                    0   
...       ...        ...                   ...                  ...   
4915        0          0                     0                    0   
4916        0          1                     0                    0   
4917        0          0                     0                    0   
4918        0          1                     0                    0   
4919        0          1                     0                    0   

      shivering  chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  \
0             0       0           0             0        0                 0   
1             0       0           0             0        0                 0   
2             0       0           0             0        0                 0   
3             0       0           0             0        0                 0   
4             0       0           0             0        0                 0   
...         ...     ...         ...           ...      ...               ...   
4915          0       0           0             0        0                 0   
4916          0       0           0             0        0                 0   
4917          0       0           0             0        0                 0   
4918          0       0           1             0        0                 0   
4919          0       0           0             0        0                 0   

      ...  pus_filled_pimples  blackheads  scurring  skin_peeling  \
0     ...                   0           0         0             0   
1     ...                   0           0         0             0   
2     ...                   0           0         0             0   
3     ...                   0           0         0             0   
4     ...                   0           0         0             0   
...   ...                 ...         ...       ...           ...   
4915  ...                   0           0         0             0   
4916  ...                   1           1         1             0   
4917  ...                   0           0         0             0   
4918  ...                   0           0         0             1   
4919  ...                   0           0         0             0   

      silver_like_dusting  small_dents_in_nails  inflammatory_nails  blister  \
0                       0                     0                   0        0   
1                       0                     0                   0        0   
2                       0                     0                   0        0   
3                       0                     0                   0        0   
4                       0                     0                   0        0   
...                   ...                   ...                 ...      ...   
4915                    0                     0                   0        0   
4916                    0                     0                   0        0   
4917                    0                     0                   0        0   
4918                    1                     1                   1        0   
4919                    0                     0                   0        1   

      red_sore_around_nose  yellow_crust_ooze  
0                        0                  0  
1                        0                  0  
2                        0                  0  
3                        0                  0  
4                        0                  0  
...                    ...                ...  
4915                     0                  0  
4916              

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [15]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3444, 132), (1476, 132), (3444,), (1476,))

In [16]:
X_train

itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  \
682         0          0                     0                    0   
3963        0          0                     0                    0   
3126        0          0                     0                    0   
4480        0          0                     0                    0   
3411        0          0                     0                    0   
...       ...        ...                   ...                  ...   
4426        0          1                     0                    0   
466         0          0                     0                    0   
3092        0          1                     0                    0   
3772        1          1                     1                    0   
860         1          1                     0                    0   

      shivering  chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  \
682           0       1           0             0        0                 0   
3963          0       1           0             0        0                 0   
3126          0       0           0             0        0                 0   
4480          0       0           0             0        1                 0   
3411          0       0           0             0        0                 0   
...         ...     ...         ...           ...      ...               ...   
4426          0       0           1             0        0                 0   
466           0       0           0             0        0                 0   
3092          0       1           1             0        0                 0   
3772          0       0           0             0        0                 0   
860           0       0           0             1        0                 0   

      ...  pus_filled_pimples  blackheads  scurring  skin_peeling  \
682   ...                   0           0         0             0   
3963  ...                   0           0         0             0   
3126  ...                   0           0         0             0   
4480  ...                   0           0         0             0   
3411  ...                   0           0         0             0   
...   ...                 ...         ...       ...           ...   
4426  ...                   0           0         0             1   
466   ...                   0           0         0             0   
3092  ...                   0           0         0             0   
3772  ...                   0           0         0             0   
860   ...                   0           0         0             0   

      silver_like_dusting  small_dents_in_nails  inflammatory_nails  blister  \
682                     0                     0                   0        0   
3963                    0                     0                   0        0   
3126                    0                     0                   0        0   
4480                    0                     0                   0        0   
3411                    0                     0                   0        0   
...                   ...                   ...                 ...      ...   
4426                    1                     1                   1        0   
466                     0                     0                   0        0   
3092                    0                     0                   0        0   
3772                    0                     0                   0        0   
860                     0                     0                   0        0   

      red_sore_around_nose  yellow_crust_ooze  
682                      0                  0  
3963                     0                  0  
3126                     0                  0  
4480                     0                  0  
3411                     0                  0  
...                    ...                ...  
4426                     0                  0  
466               

In [17]:
X_test

itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  \
373         0          1                     0                    0   
4916        0          1                     0                    0   
1550        0          0                     0                    0   
3081        0          0                     0                    0   
3857        1          0                     0                    0   
...       ...        ...                   ...                  ...   
3649        1          1                     1                    0   
1694        0          0                     0                    0   
4767        0          0                     0                    0   
3721        0          0                     0                    0   
2222        0          1                     0                    0   

      shivering  chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  \
373           0       0           0             0        0                 0   
4916          0       0           0             0        0                 0   
1550          0       0           0             0        0                 0   
3081          0       0           0             0        0                 0   
3857          0       0           0             0        0                 0   
...         ...     ...         ...           ...      ...               ...   
3649          0       0           0             0        0                 0   
1694          0       0           0             0        0                 0   
4767          0       0           0             0        1                 0   
3721          0       0           0             0        0                 0   
2222          0       1           1             0        0                 0   

      ...  pus_filled_pimples  blackheads  scurring  skin_peeling  \
373   ...                   0           1         1             0   
4916  ...                   1           1         1             0   
1550  ...                   0           0         0             0   
3081  ...                   0           0         0             0   
3857  ...                   0           0         0             0   
...   ...                 ...         ...       ...           ...   
3649  ...                   0           0         0             0   
1694  ...                   0           0         0             0   
4767  ...                   0           0         0             0   
3721  ...                   0           0         0             0   
2222  ...                   0           0         0             0   

      silver_like_dusting  small_dents_in_nails  inflammatory_nails  blister  \
373                     0                     0                   0        0   
4916                    0                     0                   0        0   
1550                    0                     0                   0        0   
3081                    0                     0                   0        0   
3857                    0                     0                   0        0   
...                   ...                   ...                 ...      ...   
3649                    0                     0                   0        0   
1694                    0                     0                   0        0   
4767                    0                     0                   0        0   
3721                    0                     0                   0        0   
2222                    0                     0                   0        0   

      red_sore_around_nose  yellow_crust_ooze  
373                      0                  0  
4916                     0                  0  
1550                     0                  0  
3081                     0                  0  
3857                     0                  0  
...                    ...                ...  
3649                     0                  0  
1694              

# Training top models

In [18]:
# Create a dictionary to store models
models = {
    "SVC": SVC(kernel='linear'),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "MultinomialNB": MultinomialNB(),
    "KNeighbors": KNeighborsClassifier(n_neighbors=5)
}

# Loop through the models, train, test, and print results
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Test the model
    predictions = model.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, predictions)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, predictions)
    
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} Confusion Matrix: {cm}")
    print(np.array2string(cm, separator=","))
   

SVC Accuracy: 1.0
SVC Confusion Matrix: [[32  0  0 ...  0  0  0]
 [ 0 39  0 ...  0  0  0]
 [ 0  0 41 ...  0  0  0]
 ...
 [ 0  0  0 ... 36  0  0]
 [ 0  0  0 ...  0 37  0]
 [ 0  0  0 ...  0  0 39]]
[[32, 0, 0,..., 0, 0, 0],
 [ 0,39, 0,..., 0, 0, 0],
 [ 0, 0,41,..., 0, 0, 0],
 ...,
 [ 0, 0, 0,...,36, 0, 0],
 [ 0, 0, 0,..., 0,37, 0],
 [ 0, 0, 0,..., 0, 0,39]]
RandomForest Accuracy: 1.0
RandomForest Confusion Matrix: [[32  0  0 ...  0  0  0]
 [ 0 39  0 ...  0  0  0]
 [ 0  0 41 ...  0  0  0]
 ...
 [ 0  0  0 ... 36  0  0]
 [ 0  0  0 ...  0 37  0]
 [ 0  0  0 ...  0  0 39]]
[[32, 0, 0,..., 0, 0, 0],
 [ 0,39, 0,..., 0, 0, 0],
 [ 0, 0,41,..., 0, 0, 0],
 ...,
 [ 0, 0, 0,...,36, 0, 0],
 [ 0, 0, 0,..., 0,37, 0],
 [ 0, 0, 0,..., 0, 0,39]]
GradientBoosting Accuracy: 1.0
GradientBoosting Confusion Matrix: [[32  0  0 ...  0  0  0]
 [ 0 39  0 ...  0  0  0]
 [ 0  0 41 ...  0  0  0]
 ...
 [ 0  0  0 ... 36  0  0]
 [ 0  0  0 ...  0 37  0]
 [ 0  0  0 ...  0  0 39]]
[[32, 0, 0,..., 0, 0, 0],
 [ 0,39, 0,..., 0,

# Single Prediction

# SVC

In [19]:
# selecting svc
svc = SVC(kernel='linear')
svc.fit(X_train,y_train)
ypred = svc.predict(X_test)
accuracy = accuracy_score(y_test , ypred)
accuracy

1.0

In [20]:
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 100.00%


# RF

In [21]:
rf = RandomForestClassifier(n_estimators= 100 , random_state=42)
rf.fit(X_train , y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test , y_pred)
accuracy

1.0

In [22]:
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 100.00%


In [23]:
# save svc
import pickle
pickle.dump(rf, open('model/rf.pkl','wb') )# for RF model 
pickle.dump(svc , open('model/svc.pkl','wb') )# for  Svc model 

# Load Model

In [24]:
svc = pickle.load(open("model/svc.pkl", 'rb'))
rf = pickle.load(open("model/rf.pkl", 'rb'))

In [25]:
svc

SVC(kernel='linear')

In [26]:
rf

RandomForestClassifier(random_state=42)

# Test 01

In [27]:
print("Pridicted Label :", svc.predict(X_test.iloc[0].values.reshape(1, -1)))
print("Actual Label :", y_test[0])

Pridicted Label : [2]
Actual Label : 2


# Test 02

In [28]:
print("Pridicted Label :", svc.predict(X_test.iloc[10].values.reshape(1, -1)))
print("Actual Label :", y_test[10])

Pridicted Label : [38]
Actual Label : 38


# Recommendation Systerm and Prediction
# Logic For Reccommendations

# Symptom data set

In [29]:
symtoms_df.head(5)

Unnamed: 0           Disease   Symptom_1              Symptom_2  \
0           0  Fungal infection     itching              skin_rash   
1           1  Fungal infection   skin_rash   nodal_skin_eruptions   
2           2  Fungal infection     itching   nodal_skin_eruptions   
3           3  Fungal infection     itching              skin_rash   
4           4  Fungal infection     itching              skin_rash   

               Symptom_3             Symptom_4  
0   nodal_skin_eruptions   dischromic _patches  
1    dischromic _patches                   NaN  
2    dischromic _patches                   NaN  
3    dischromic _patches                   NaN  
4   nodal_skin_eruptions                   NaN

In [30]:
Symptom_severity_df.head(3)

Symptom  weight
0               itching       1
1             skin_rash       3
2  nodal_skin_eruptions       4

# Precautions Dataset

In [31]:
precautions_df.head(5)

Unnamed: 0         Disease                      Precaution_1  \
0           0   Drug Reaction                   stop irritation   
1           1         Malaria          Consult nearest hospital   
2           2         Allergy                    apply calamine   
3           3  Hypothyroidism                     reduce stress   
4           4       Psoriasis  wash hands with warm soapy water   

                   Precaution_2        Precaution_3  \
0      consult nearest hospital    stop taking drug   
1               avoid oily food  avoid non veg food   
2       cover area with bandage                 NaN   
3                      exercise         eat healthy   
4  stop bleeding using pressure      consult doctor   

                  Precaution_4  
0                    follow up  
1           keep mosquitos out  
2  use ice to compress itching  
3             get proper sleep  
4                   salt baths

# workout Dataset

In [32]:
workout_df.head(3)

Unnamed: 0  Unnamed: 0.1           disease                    workout
0           0             0  Fungal infection         Avoid sugary foods
1           1             1  Fungal infection         Consume probiotics
2           2             2  Fungal infection  Increase intake of garlic

# Medications data 

In [33]:
medications_df.head(5)

Disease                                         Medication
0     Fungal infection  ['Antifungal Cream', 'Fluconazole', 'Terbinafi...
1              Allergy  ['Antihistamines', 'Decongestants', 'Epinephri...
2                 GERD  ['Proton Pump Inhibitors (PPIs)', 'H2 Blockers...
3  Chronic cholestasis  ['Ursodeoxycholic acid', 'Cholestyramine', 'Me...
4        Drug Reaction  ['Antihistamines', 'Epinephrine', 'Corticoster...

# Description Data 

In [34]:
description_df.head(5)

Disease                                        Description
0     Fungal infection  Fungal infection is a common skin condition ca...
1              Allergy  Allergy is an immune system reaction to a subs...
2                 GERD  GERD (Gastroesophageal Reflux Disease) is a di...
3  Chronic cholestasis  Chronic cholestasis is a condition where bile ...
4        Drug Reaction  Drug Reaction occurs when the body reacts adve...

# Diets Data

In [35]:
diets_df.head(5)

Disease                                               Diet
0     Fungal infection  ['Antifungal Diet', 'Probiotics', 'Garlic', 'C...
1              Allergy  ['Elimination Diet', 'Omega-3-rich foods', 'Vi...
2                 GERD  ['Low-Acid Diet', 'Fiber-rich foods', 'Ginger'...
3  Chronic cholestasis  ['Low-Fat Diet', 'High-Fiber Diet', 'Lean prot...
4        Drug Reaction  ['Antihistamine Diet', 'Omega-3-rich foods', '...

# Model Prediction Function 

# Define the synonym dictionary for symptoms mapping and user friendly words for user friendly Model 

In [36]:
#============================================================
# custome and helping functions
#==========================helper funtions================

def helper(dis):
    desc = description_df[description_df['Disease'] == dis]['Description']
    desc = " ".join([w for w in desc])

    pre = precautions_df[precautions_df['Disease'] == dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']]
    pre = [col for col in pre.values]

    med = medications_df[medications_df['Disease'] == dis]['Medication']
    med = [med for med in med.values]

    die = diets_df[diets_df['Disease'] == dis]['Diet']
    die = [die for die in die.values]

    wrkout = workout_df[workout_df['disease'] == dis] ['workout']

    return desc,pre,med,die,wrkout


symptoms_dict = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3, 'shivering': 4, 'chills': 5, 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8, 'ulcers_on_tongue': 9, 'muscle_wasting': 10, 'vomiting': 11, 'burning_micturition': 12, 'spotting_urination': 13, 'fatigue': 14, 'weight_gain': 15, 'anxiety': 16, 'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20, 'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24, 'high_fever': 25, 'sunken_eyes': 26, 'breathlessness': 27, 'sweating': 28, 'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32, 'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36, 'back_pain': 37, 'constipation': 38, 'abdominal_pain': 39, 'diarrhoea': 40, 'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 'acute_liver_failure': 44, 'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48, 'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52, 'sinus_pressure': 53, 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56, 'weakness_in_limbs': 57, 'fast_heart_rate': 58, 'pain_during_bowel_movements': 59, 'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 'neck_pain': 63, 'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68, 'swollen_blood_vessels': 69, 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71, 'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74, 'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77, 'knee_pain': 78, 'hip_joint_pain': 79, 'muscle_weakness': 80, 'stiff_neck': 81, 'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84, 'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87, 'loss_of_smell': 88, 'bladder_discomfort': 89, 'foul_smell_of urine': 90, 'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93, 'toxic_look_(typhos)': 94, 'depression': 95, 'irritability': 96, 'muscle_pain': 97, 'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100, 'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103, 'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107, 'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110, 'receiving_blood_transfusion': 111, 'receiving_unsterile_injections': 112, 'coma': 113, 'stomach_bleeding': 114, 'distention_of_abdomen': 115, 'history_of_alcohol_consumption': 116, 'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119, 'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123, 'scurring': 124, 'skin_peeling': 125, 'silver_like_dusting': 126, 'small_dents_in_nails': 127, 'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 'yellow_crust_ooze': 131}
diseases_list = {15: 'Fungal infection', 4: 'Allergy', 16: 'GERD', 9: 'Chronic cholestasis', 14: 'Drug Reaction', 33: 'Peptic ulcer diseae', 1: 'AIDS', 12: 'Diabetes ', 17: 'Gastroenteritis', 6: 'Bronchial Asthma', 23: 'Hypertension ', 30: 'Migraine', 7: 'Cervical spondylosis', 32: 'Paralysis (brain hemorrhage)', 28: 'Jaundice', 29: 'Malaria', 8: 'Chicken pox', 11: 'Dengue', 37: 'Typhoid', 40: 'hepatitis A', 19: 'Hepatitis B', 20: 'Hepatitis C', 21: 'Hepatitis D', 22: 'Hepatitis E', 3: 'Alcoholic hepatitis', 36: 'Tuberculosis', 10: 'Common Cold', 34: 'Pneumonia', 13: 'Dimorphic hemmorhoids(piles)', 18: 'Heart attack', 39: 'Varicose veins', 26: 'Hypothyroidism', 24: 'Hyperthyroidism', 25: 'Hypoglycemia', 31: 'Osteoarthristis', 5: 'Arthritis', 0: '(vertigo) Paroymsal  Positional Vertigo', 2: 'Acne', 38: 'Urinary tract infection', 35: 'Psoriasis', 27: 'Impetigo'}


# Spelling correction for incorrect symptoms entered by the user mistakenly

In [37]:
# # Initialize the TextBlob object for spelling correction
def correct_spelling(symptom):
    # Correct the spelling of a single symptom
    blob = TextBlob(symptom)
    return str(blob.correct())

# Adding more user-friendly words and synonyms to the dictionary for a better user experience

In [38]:
symptom_mapping = defaultdict(lambda: "unknown", {
    "itching": ["itching"],
    "skin_rash": ["skin rash", "rash", "dermatitis"],
    "nodal_skin_eruptions": ["nodal skin eruptions", "skin eruptions", "bumps"],
    "continuous_sneezing": ["continuous sneezing", "sneezing"],
    "shivering": ["shivering", "trembling"],
    "chills": ["chills", "cold sensation", "cold"],
    "joint_pain": ["joint pain", "arthralgia", "aching joints"],
    "stomach_pain": ["stomach pain", "abdominal pain", "belly ache"],
    "acidity": ["acidity", "heartburn", "acid reflux"],
    "ulcers_on_tongue": ["ulcers on tongue", "tongue ulcers", "mouth sores"],
    "muscle_wasting": ["muscle wasting", "muscle loss"],
    "vomiting": ["vomiting", "emesis", "throwing up"],
    "burning_micturition": ["burning micturition", "burning urination", "painful urination"],
    "spotting_urination": ["spotting urination", "blood in urine", "hematuria"],
    "fatigue": ["fatigue", "tiredness", "exhaustion"],
    "weight_gain": ["weight gain", "increased weight"],
    "anxiety": ["anxiety", "nervousness", "worry"],
    "cold_hands_and_feets": ["cold hands and feet", "cold extremities"],
    "mood_swings": ["mood swings", "emotional changes"],
    "weight_loss": ["weight loss", "decreased weight"],
    "restlessness": ["restlessness", "agitation"],
    "lethargy": ["lethargy", "sluggishness"],
    "patches_in_throat": ["patches in throat", "throat patches", "throat lesions"],
    "irregular_sugar_level": ["irregular sugar level", "unstable glucose", "blood sugar fluctuations"],
    "cough": ["cough", "coughing"],
    "high_fever": ["high fever", "elevated temperature"],
    "sunken_eyes": ["sunken eyes", "hollow eyes"],
    "breathlessness": ["breathlessness", "shortness of breath", "dyspnea"],
    "sweating": ["sweating", "perspiration"],
    "dehydration": ["dehydration", "fluid loss"],
    "indigestion": ["indigestion", "upset stomach"],
    "headache": ["headache", "head pain", "migraine"],
    "yellowish_skin": ["yellowish skin", "jaundice"],
    "dark_urine": ["dark urine"],
    "nausea": ["nausea", "queasiness"],
    "loss_of_appetite": ["loss of appetite", "no appetite", "anorexia"],
    "pain_behind_the_eyes": ["pain behind the eyes", "eye pain"],
    "back_pain": ["back pain", "lower back pain"],
    "constipation": ["constipation", "difficulty passing stool"],
    "abdominal_pain": ["abdominal pain", "belly pain", "stomach ache"],
    "diarrhoea": ["diarrhoea", "loose stools"],
    "mild_fever": ["mild fever", "fever" "low-grade fever"],
    "yellow_urine": ["yellow urine"],
    "yellowing_of_eyes": ["yellowing of eyes", "scleral icterus"],
    "acute_liver_failure": ["acute liver failure", "hepatic failure"],
    "fluid_overload": ["fluid overload", "edema"],
    "swelling_of_stomach": ["swelling of stomach", "abdominal bloating"],
    "swelled_lymph_nodes": ["swelled lymph nodes", "enlarged lymph nodes"],
    "malaise": ["malaise", "general discomfort"],
    "blurred_and_distorted_vision": ["blurred and distorted vision", "blurry vision"],
    "phlegm": ["phlegm", "mucus"],
    "throat_irritation": ["throat irritation", "sore throat"],
    "redness_of_eyes": ["redness of eyes", "bloodshot eyes"],
    "sinus_pressure": ["sinus pressure", "sinus congestion"],
    "runny_nose": ["runny nose", "rhinorrhea"],
    "congestion": ["congestion", "nasal blockage"],
    "chest_pain": ["chest pain", "angina"],
    "weakness_in_limbs": ["weakness in limbs", "limb weakness"],
    "fast_heart_rate": ["fast heart rate", "tachycardia"],
    "pain_during_bowel_movements": ["pain during bowel movements", "painful defecation"],
    "pain_in_anal_region": ["pain in anal region", "anal pain"],
    "bloody_stool": ["bloody stool", "rectal bleeding"],
    "irritation_in_anus": ["irritation in anus", "anal itching"],
    "neck_pain": ["neck pain", "cervical pain"],
    "dizziness": ["dizziness", "lightheadedness"],
    "cramps": ["cramps", "muscle cramps", "spasms"],
    "bruising": ["bruising", "hematoma"],
    "obesity": ["obesity", "overweight"],
    "swollen_legs": ["swollen legs", "leg edema"],
    "swollen_blood_vessels": ["swollen blood vessels", "varicose veins"],
    "puffy_face_and_eyes": ["puffy face and eyes", "facial swelling"],
    "enlarged_thyroid": ["enlarged thyroid", "goiter"],
    "brittle_nails": ["brittle nails", "weak nails"],
    "swollen_extremeties": ["swollen extremities", "swollen arms and legs"],
    "excessive_hunger": ["excessive hunger", "polyphagia"],
    "extra_marital_contacts": ["extra marital contacts", "multiple sexual partners"],
    "drying_and_tingling_lips": ["drying and tingling lips", "lip dryness"],
    "slurred_speech": ["slurred speech", "dysarthria"],
    "knee_pain": ["knee pain", "pain in the knees"],
    "hip_joint_pain": ["hip joint pain", "hip pain"],
    "muscle_weakness": ["muscle weakness", "muscle fatigue"],
    "stiff_neck": ["stiff neck", "neck stiffness"],
    "swelling_joints": ["swelling joints", "joint swelling"],
    "movement_stiffness": ["movement stiffness", "rigidity"],
    "spinning_movements": ["spinning movements", "vertigo"],
    "loss_of_balance": ["loss of balance", "balance problems"],
    "unsteadiness": ["unsteadiness", "lack of balance"],
    "weakness_of_one_body_side": ["weakness of one body side", "hemiparesis"],
    "loss_of_smell": ["loss of smell", "anosmia"],
    "bladder_discomfort": ["bladder discomfort", "bladder pain"],
    "foul_smell_of_urine": ["foul smell of urine", "smelly urine"],
    "continuous_feel_of_urine": ["continuous feel of urine", "urgency to urinate"],
    "passage_of_gases": ["passage of gases", "flatulence"],
    "internal_itching": ["internal itching"],
    "toxic_look_(typhos)": ["toxic look (typhos)", "septic appearance"],
    "depression": ["depression", "low mood"],
    "irritability": ["irritability", "easily annoyed"],
    "muscle_pain": ["muscle pain", "myalgia"],
    "altered_sensorium": ["altered sensorium", "confusion"],
    "red_spots_over_body": ["red spots over body", "rash with red spots"],
    "belly_pain": ["belly pain", "abdominal pain"],
    "abnormal_menstruation": ["abnormal menstruation", "irregular periods"],
    "dischromic_patches": ["dischromic patches", "skin discoloration"],
    "watering_from_eyes": ["watering from eyes", "teary eyes"],
    "increased_appetite": ["increased appetite", "hyperphagia"],
    "polyuria": ["polyuria", "excessive urination"],
    "family_history": ["family history", "genetic predisposition"],
    "mucoid_sputum": ["mucoid sputum", "mucus in sputum"],
    "rusty_sputum": ["rusty sputum", "blood-tinged sputum"],
    "lack_of_concentration": ["lack of concentration", "difficulty focusing"],
    "visual_disturbances": ["visual disturbances", "vision problems"],
    "receiving_blood_transfusion": ["receiving blood transfusion"],
    "receiving_unsterile_injections": ["receiving unster"]
    # Add more symptoms and their synonyms here
})

# Auto correction for Spelling mistakes and synonums for Prediction class 

In [39]:
def get_predicted_value(patient_symptoms):
    input_vector = np.zeros(len(symptoms_dict))
    
    # Print the initial input vector
    print("Initial input vector:", input_vector)
    
    for item in patient_symptoms:
        # First, check if the symptom is in the symptoms_dict
        corrected_item = item  # Initial assumption, might be corrected later
        index = symptoms_dict.get(corrected_item, -1)
        
        if index != -1:
            # If found in symptoms_dict
            input_vector[index] = 1
            print(f"Symptom '{item}' is mapped to index {index} directly from symptoms_dict")
        else:
            # Check if the symptom is in the symptom_mapping
            found = False
            for key, synonyms in symptom_mapping.items():
                if item in synonyms:
                    index = symptoms_dict.get(key, -1)
                    if index != -1:
                        input_vector[index] = 1
                        print(f"Symptom '{item}' is mapped to synonym '{key}' with index {index}")
                        found = True
                        break
            
            if not found:
                # Correct spelling if not found in either dictionary
                corrected_item = correct_spelling(item)
                index = symptoms_dict.get(corrected_item, -1)
                
                if index != -1:
                    # If found in symptoms_dict after correction
                    input_vector[index] = 1
                    print(f"Symptom '{item}' corrected to '{corrected_item}' and mapped to index {index}")
                else:
                    # Check if the corrected symptom is in the symptom_mapping
                    found = False
                    for key, synonyms in symptom_mapping.items():
                        if corrected_item in synonyms:
                            index = symptoms_dict.get(key, -1)
                            if index != -1:
                                input_vector[index] = 1
                                print(f"Symptom '{item}' corrected to '{corrected_item}' and mapped to synonym '{key}' with index {index}")
                                found = True
                                break
                    if not found:
                        print(f"Warning: Symptom '{item}' with corrected spelling '{corrected_item}' is not in the symptoms dictionary or mapping")
    
    # Print the final input vector
    print("Final input vector:", input_vector)
    
    predicted_disease_index = svc.predict([input_vector])[0]
    predicted_disease = diseases_list.get(predicted_disease_index, "Unknown Disease")
    
    # Print the predicted disease
    print(f"Predicted disease index: {predicted_disease_index}")
    print(f"Predicted disease: {predicted_disease}")
    
    # return predicted_disease
    return diseases_list[svc.predict([input_vector])[0]]


In [40]:
# def get_predicted_value(patient_symptoms):
#     input_vector = np.zeros(len(symptoms_dict))
    
#     # Print the initial input vector
#     print("Initial input vector:", input_vector)
    
#     for item in patient_symptoms:
#         # Check if the symptom is in the dictionary
#         if item in symptoms_dict:
#             index = symptoms_dict[item]
#             input_vector[index] = 1
#             print(f"Symptom '{item}' mapped to index {index}")
#         else:
#             # If not found, check the symptom_mapping
#             primary_symptom = None
#             for key, synonyms in symptom_mapping.items():
#                 if item in synonyms:
#                     primary_symptom = key
#                     break
            
#             if primary_symptom:
#                 if primary_symptom in symptoms_dict:
#                     index = symptoms_dict[primary_symptom]
#                     input_vector[index] = 1
#                     print(f"Symptom '{item}' (mapped to '{primary_symptom}') set at index {index} via symptom_mapping")
#                 else:
#                     print(f"Warning: Mapped symptom '{primary_symptom}' is not in the symptoms_dict")
#             else:
#                 print(f"Warning: Symptom '{item}' is not mapped to any known primary symptom")
    
#     # print(f"Warning: Symptom '{item}' is not in the symptoms dictionary")
    
#     # Print the final input vector
#     print("Final input vector:", input_vector)
    
#     predicted_disease_index = svc.predict([input_vector])[0]
#     predicted_disease = diseases_list.get(predicted_disease_index, "Unknown Disease")
    
#     # Print the predicted disease
#     print(f"Predicted disease index: {predicted_disease_index}")
#     print(f"Predicted disease: {predicted_disease}")
    
#     return predicted_disease


# Test 1

# Adding an input function to allow the user to enter multiple symptoms for prediction.

In [41]:
# Main code 
symptoms = input("Enter your symptom/symptoms with using coma for e.g cold, Stress, vomiting, painful urination, genetic predisposition, multiple sexual partners .......").lower()
user_symptoms = [s.strip() for s in symptoms.split(',')]

# Remove any extra characters, if any
user_symptoms = [sym.strip("[]' ") for sym in user_symptoms]
predicted_disease = get_predicted_value(user_symptoms)

desc, pre, med, die, wrkout = helper(predicted_disease)

Initial input vector: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Symptom 'vommiting' corrected to 'vomiting' and mapped to index 11
Symptom 'blood in urine' is mapped to synonym 'spotting_urination' with index 13
Symptom 'painful urination' is mapped to synonym 'burning_micturition' with index 12
Final input vector: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [42]:
symptoms

'vommiting, blood in urine, painful urination'

In [43]:
predicted_disease

'Drug Reaction'

# Prediction/Result Print

In [44]:
print("=================predicted disease============")
print(predicted_disease)
print("=================description==================")
print(desc)
print("=================precautions==================")
i = 1
for p_i in pre[0]:
    print(i, ": ", p_i)
    i += 1

print("=================medications==================")
for m_i in med:
    print(i, ": ", m_i)
    i += 1

print("=================workout==================")
for w_i in wrkout:
    print(i, ": ", w_i)
    i += 1

print("=================diets==================")
for d_i in die:
    print(i, ": ", d_i)
    i += 1

=================predicted disease============
Drug Reaction
=================description==================
Drug Reaction occurs when the body reacts adversely to a medication.
=================precautions==================
1 :  stop irritation
2 :  consult nearest hospital
3 :  stop taking drug
4 :  follow up
=================medications==================
5 :  ['Antihistamines', 'Epinephrine', 'Corticosteroids', 'Antibiotics', 'Antifungal Cream']
=================workout==================
6 :  Discontinue offending medication
7 :  Stay hydrated
8 :  Include anti-inflammatory foods
9 :  Consume antioxidants
10 :  Avoid trigger foods
11 :  Include omega-3 fatty acids
12 :  Limit caffeine and alcohol
13 :  Stay hydrated
14 :  Eat a balanced diet
15 :  Consult a healthcare professional
=================diets==================
16 :  ['Antihistamine Diet', 'Omega-3-rich foods', 'Vitamin C-rich foods', 'Quercetin-rich foods', 'Probiotics']


# To Run ML MODEL OUTPUT/Flask API (Dashboard) Run "main.py" file 